In [1]:
spark.sql("""
  SELECT 
    ew1.*,
    COALESCE(age.Age_Multiplier, 1) AS Age_Multiplier,
    COALESCE(acc.Accuracy_Multiplier, 1) AS Accuracy_Multiplier
  FROM
    elementweightage ew1
    INNER JOIN (
      SELECT 
        ew2.group,
        ROUND(1 / SUM(ew2.age_scope * ew2.weightage), 5) AS Age_Multiplier
      FROM 
        elementweightage ew2
      GROUP BY group
      ) AS age ON age.group = ew1.group
    INNER JOIN (
      SELECT
        ew3.group,
        ROUND(1 / SUM(ew3.accuracy_scope * ew3.weightage), 5) AS Accuracy_Multiplier
      FROM
        elementweightage ew3
      GROUP BY group
    ) AS acc ON acc.group = ew1.group
""").createOrReplaceTempView("elementWeightage")

spark.sql("""
SELECT
  Length_Ft,
  BusinessUnit AS Company_Code,
  REGEXP_REPLACE(Route, " .*", "") AS Route,
  REGEXP_REPLACE(Route, ".* ", "") AS Route_Section,
  ValveSectionName AS Valve_Section,
  ExternalCorrosionRuptureFailureFrequency_PerMiYr * Safety_70_Percent AS EC_Risk,
  ExternalCorrosionRuptureFailureFrequency_PerMiYr AS Probability_of_Failure,
  Safety_70_Percent AS Consequence,
  Risk * Safety_70_Percent AS Total_Risk
FROM riskrunbyvalvesection
""").createOrReplaceTempView("ValveSectionRiskRun")

spark.sql("""
SELECT
  Length_Ft,
  BusinessUnit AS Company_Code,
  REGEXP_REPLACE(Route, " .*", "") AS Route,
  REGEXP_REPLACE(Route, ".* ", "") AS Route_Section,
  ExternalCorrosionRuptureFailureFrequency_PerMiYr * Safety_70_Percent AS EC_Risk,
  ExternalCorrosionRuptureFailureFrequency_PerMiYr AS Probability_of_Failure,
  Safety_70_Percent AS Consequence,
  Risk * Safety_70_Percent AS Total_Risk
FROM riskrunbyroute
""").createOrReplaceTempView("RouteRiskRun")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-132252061730641> in <module> () 
 22 GROUP BY group 
 23 ) AS acc ON acc.group = ew1.group
 ---> 24 """).createOrReplaceTempView("elementWeightage")
 25 
 26 spark.sql("""

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 827 [ Row ( f1 = 1 , f2 = u'row1' ) , Row ( f1 = 2 , f2 = u'row2' ) , Row ( f1 = 3 , f2 = u'row3' ) ] 
 828 """
 --> 829 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 830 
 831 @ since ( 2.0 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'Table or view not found: elementweightage; line 7 pos 4'

In [2]:
add_route_information_to_table("CPSurveyReading", "seriesid", "RouteCPSurveyReadingDate")
add_route_information_to_table("HydroTest", "seriesidbegin", "RouteHydroTestDate")
add_route_information_to_table("HCACFR", "seriesidbegin", "RouteHCA")
add_route_information_to_table("MOP", "seriesidbegin", "RouteMOP")
add_route_information_to_table("PipeSegment", "seriesidbegin", "RoutePipeSeg")

add_route_information_to_two_tables("TestPostPoint", "TestPostPointReading", "seriesid", "testpostpointID", "testpostpointID", "RouteTestPostPointDate")

In [3]:
spark.sql("""
  SELECT 
    COALESCE(Company_Code, '') AS Company_Code,
    Company_Description,
    COALESCE(Segment_Name, '') AS Segment_Name,
    CONCAT(segment_name, '/', line) AS Route,
    Area_Name,
    Region_Name,
    Line,
    Route_Section,
    Valve_Section,
    Measure_Begin,
    Measure_End
  FROM
    Hierarchy
""").createOrReplaceTempView("Hierarchy")

In [4]:
spark.sql("""
select 
  REGEXP_REPLACE(LineName, " .*", "") AS route,
  REGEXP_REPLACE(LineName, ".* ", "") AS route_section,
  r.ILIRStartDate, 
  r.id, 
  r.BeginStationSeriesId,
  r.BeginStationNum,
  r.EndStationSeriesId,
  r.EndStationNum,
  o.OperatingUnitName AS Company_Code
from 
  inlineinspectionRange r
  join iras_p_StationSeries s on s.id = r.BeginStationSeriesId
  join iras_p_LineLoop ll on ll.id = s.LineLoopId
  join iras_p_PipelineSystem p on ll.PipelineSystemId = p.id
  join OperatingUnit o on O.id = p.OperatingUnitId
""").createOrReplaceTempView("RouteILI")